# Forex Signal V15: Multi-Timeframe Analysis (1m, 5m, 15m, 30m, 1h)

This notebook implements a multi-timeframe machine learning approach for Forex prediction.
It aggregates features from 1-minute, 5-minute, 15-minute, 30-minute, and 1-hour data to predict price movements.

**Key Features:**
- **Multi-Timeframe Resampling:** Converts 1m data into 5m, 15m, 30m, 1h candles.
- **MTF Indicators:** Calculates RSI, MACD, BB, EMA for *all* timeframes.
- **Feature Merging:** Aligns higher timeframe features to the 1-minute timeline (using forward fill to avoid lookahead bias).
- **Ensemble Model:** Trains XGBoost, LightGBM, and CatBoost on the combined dataset.

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import joblib
import time
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# Paths
BASE_DIR = Path(r'c:\Users\Acer\Desktop\Forex-Signal-App')
DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models' / 'signal_generator_v15'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Constants
TIMEFRAMES = ['1min', '5min', '15min', '30min', '1h']
TARGET_TIMEFRAME = '1min'  # We predict on 1-min data using MTF context
FORWARD_BARS = 15          # Predict 15 minutes into the future
TP_PIPS = 10
SL_PIPS = 10

print(f"🚀 FOREX SIGNAL V15: Multi-Timeframe Analysis")
print(f"✓ Model Directory: {MODEL_DIR}")

🚀 FOREX SIGNAL V15: Multi-Timeframe Analysis
✓ Model Directory: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v15


In [9]:
# 1. Load Data (Train & Test Split)
def load_tf_data(timeframe, dataset_type='train'):
    """
    Load specific timeframe data from train or test folder
    """
    # File names are like EURUSD_m1.csv
    filename = f'EURUSD_{timeframe}.csv'
    file_path = DATA_DIR / dataset_type / filename
        
    if not file_path.exists():
        print(f"⚠️ Warning: {file_path} not found.")
        return None
        
    # print(f"Loading {dataset_type} {timeframe} data...")
    df = pd.read_csv(file_path)
    
    # Standardize columns
    df.columns = [c.lower() for c in df.columns]
    
    # Ensure time column is datetime
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'])
        df.set_index('time', inplace=True)
        df.sort_index(inplace=True)
    
    return df

print("Data loader defined.")

Data loader defined.


In [10]:
# 2. Load Multi-Timeframe Data
timeframes_map = {
    '1min': 'm1',
    '5min': 'm5',
    '15min': 'm15',
    '30min': 'm30',
    '1h': 'h1'
}

dfs_train = {}
dfs_test = {}

print("Loading TRAIN data (2015-2023)...")
for tf_name, tf_code in timeframes_map.items():
    dfs_train[tf_name] = load_tf_data(tf_code, 'train')
    if dfs_train[tf_name] is not None:
        print(f"  {tf_name}: {len(dfs_train[tf_name]):,} rows")

print("\nLoading TEST data (2024-2025)...")
for tf_name, tf_code in timeframes_map.items():
    dfs_test[tf_name] = load_tf_data(tf_code, 'test')
    if dfs_test[tf_name] is not None:
        print(f"  {tf_name}: {len(dfs_test[tf_name]):,} rows")

Loading TRAIN data (2015-2023)...
  1min: 3,354,904 rows
  5min: 671,581 rows
  15min: 224,382 rows
  30min: 112,194 rows
  1h: 56,098 rows

Loading TEST data (2024-2025)...
  1min: 743,476 rows
  5min: 148,502 rows
  15min: 49,807 rows
  30min: 24,907 rows
  1h: 12,454 rows


In [11]:
# 3. Feature Engineering
def add_technical_indicators(df):
    """
    Calculates technical indicators for a given dataframe.
    """
    if df is None or len(df) == 0:
        return None
        
    df = df.copy()
    
    # RSI (14)
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD (12, 26, 9)
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Bollinger Bands (20, 2)
    df['bb_mid'] = df['close'].rolling(window=20).mean()
    df['bb_std'] = df['close'].rolling(window=20).std()
    df['bb_upper'] = df['bb_mid'] + (df['bb_std'] * 2)
    df['bb_lower'] = df['bb_mid'] - (df['bb_std'] * 2)
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_mid']
    
    # EMAs
    for span in [9, 21, 50, 200]:
        df[f'ema_{span}'] = df['close'].ewm(span=span, adjust=False).mean()
        
    # ATR (14)
    high_low = df['high'] - df['low']
    high_close = (df['high'] - df['close'].shift()).abs()
    low_close = (df['low'] - df['close'].shift()).abs()
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    df['atr'] = true_range.rolling(14).mean()
    
    return df.dropna()

def process_all_timeframes(dfs_dict):
    processed = {}
    for tf, df in dfs_dict.items():
        if df is not None:
            # print(f"Processing {tf}...")
            processed[tf] = add_technical_indicators(df)
            
            # Rename columns with prefix (except for 1min which is base)
            if tf != '1min' and processed[tf] is not None:
                prefix = f"{tf}_"
                new_cols = {col: f"{prefix}{col}" for col in processed[tf].columns}
                processed[tf].rename(columns=new_cols, inplace=True)
    return processed

print("Processing TRAIN features...")
processed_train = process_all_timeframes(dfs_train)

print("Processing TEST features...")
processed_test = process_all_timeframes(dfs_test)

print("Feature engineering complete.")

Processing TRAIN features...
Processing TEST features...
Feature engineering complete.


In [12]:
# 4. Merge Data (Multi-Timeframe Alignment)
def merge_timeframes(processed_dfs):
    if '1min' not in processed_dfs or processed_dfs['1min'] is None:
        return None
        
    master_df = processed_dfs['1min'].copy()

    for tf in ['5min', '15min', '30min', '1h']:
        if tf in processed_dfs and processed_dfs[tf] is not None:
            # print(f"Merging {tf} features...")
            resampled_df = processed_dfs[tf].sort_index()
            
            master_df = pd.merge_asof(
                master_df,
                resampled_df,
                left_index=True,
                right_index=True,
                direction='backward'
            )
            
    master_df.dropna(inplace=True)
    return master_df

print("Merging TRAIN data...")
master_train = merge_timeframes(processed_train)
print(f"Train Shape: {master_train.shape}")

print("Merging TEST data...")
master_test = merge_timeframes(processed_test)
print(f"Test Shape: {master_test.shape}")

Merging TRAIN data...
Train Shape: (3353766, 95)
Merging TEST data...
Test Shape: (742341, 95)


In [13]:
# 5. Define Target
def create_target(df, tp_pips=10, sl_pips=10, forward_bars=15):
    targets = []
    pip_size = 0.0001
    tp = tp_pips * pip_size
    sl = sl_pips * pip_size
    
    closes = df['close'].values
    highs = df['high'].values
    lows = df['low'].values
    
    for i in range(len(df) - forward_bars):
        current_close = closes[i]
        
        # Look ahead window
        future_highs = highs[i+1 : i+1+forward_bars]
        future_lows = lows[i+1 : i+1+forward_bars]
        
        max_high = np.max(future_highs)
        min_low = np.min(future_lows)
        
        # Check for Buy Signal
        if (max_high - current_close) >= tp and (current_close - min_low) < sl:
            targets.append(1) # BUY
        # Check for Sell Signal
        elif (current_close - min_low) >= tp and (max_high - current_close) < sl:
            targets.append(2) # SELL
        else:
            targets.append(0) # HOLD
            
    # Pad the end
    targets.extend([0] * forward_bars)
    return np.array(targets)

print("Creating targets for TRAIN...")
master_train['target'] = create_target(master_train, TP_PIPS, SL_PIPS, FORWARD_BARS)
train_df = master_train.iloc[:-FORWARD_BARS] # Remove last rows

print("Creating targets for TEST...")
master_test['target'] = create_target(master_test, TP_PIPS, SL_PIPS, FORWARD_BARS)
test_df = master_test.iloc[:-FORWARD_BARS] # Remove last rows

print("Target Distribution (Train):")
print(train_df['target'].value_counts(normalize=True))

# Define feature columns
feature_cols = [c for c in train_df.columns if c not in ['target', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'volume']]
# Also exclude raw OHLC of higher timeframes
feature_cols = [c for c in feature_cols if not any(x in c for x in ['open', 'high', 'low', 'tick_volume', 'volume'])]

print(f"Training Features: {len(feature_cols)}")
print(f"Train Set: {len(train_df):,} rows")
print(f"Test Set:  {len(test_df):,} rows")

# Define paths for saving processed data
train_path = DATA_DIR / 'train' / 'EURUSD_train_processed.csv'
test_path = DATA_DIR / 'test' / 'EURUSD_test_processed.csv'

Creating targets for TRAIN...
Creating targets for TEST...
Target Distribution (Train):
target
0    0.865145
2    0.067941
1    0.066913
Name: proportion, dtype: float64
Training Features: 69
Train Set: 3,353,751 rows
Test Set:  742,326 rows


In [ ]:
# ==========================================
# SAVE & RELOAD SPLIT DATA
# ==========================================

# Use absolute paths defined earlier
train_path = DATA_DIR / 'train' / 'EURUSD_train_processed.csv'
test_path = DATA_DIR / 'test' / 'EURUSD_test_processed.csv'

print(f"Saving Train data to {train_path}...")
train_df.to_csv(train_path)

print(f"Saving Test data to {test_path}...")
test_df.to_csv(test_path)

print("Reloading data from files to ensure training uses the saved files...")
train_df = pd.read_csv(train_path, index_col=0, parse_dates=True)
test_df = pd.read_csv(test_path, index_col=0, parse_dates=True)

print("Data reloaded successfully.")
print(f"Train Shape: {train_df.shape}")
print(f"Test Shape: {test_df.shape}")

Saving Train data to c:\Users\Acer\Desktop\Forex-Signal-App\data\train\EURUSD_train_processed.csv...


In [ ]:
# 6. Train Models (XGBoost & CatBoost)

X_train = train_df[feature_cols]
y_train = train_df['target']
X_test = test_df[feature_cols]
y_test = test_df['target']

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# XGBoost
print("Training XGBoost...")
xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softprob',
    num_class=3,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train_scaled, y_train)

# CatBoost
print("Training CatBoost...")
cat_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    loss_function='MultiClass',
    verbose=100,
    random_seed=42,
    task_type='CPU' # Use GPU if available
)
cat_model.fit(X_train_scaled, y_train)

print("Training Complete.")

In [ ]:
# 7. Evaluation
def evaluate_model(model, X, y, name="Model"):
    preds = model.predict(X)
    if isinstance(preds[0], list) or (isinstance(preds, np.ndarray) and preds.ndim > 1):
         preds = np.array([p[0] for p in preds]) # Handle CatBoost output shape if needed
         
    acc = accuracy_score(y, preds)
    print(f"--- {name} Performance ---")
    print(f"Accuracy: {acc:.4f}")
    print(classification_report(y, preds))
    return acc

evaluate_model(xgb_model, X_test_scaled, y_test, "XGBoost")
evaluate_model(cat_model, X_test_scaled, y_test, "CatBoost")

# Feature Importance (XGBoost)
plt.figure(figsize=(10, 8))
xgb.plot_importance(xgb_model, max_num_features=20)
plt.title("Top 20 Features (XGBoost)")
plt.show()

In [ ]:
# 8. Save Models
print("Saving models...")
joblib.dump(xgb_model, MODEL_DIR / 'xgboost_v15.joblib')
joblib.dump(cat_model, MODEL_DIR / 'catboost_v15.joblib')
joblib.dump(scaler, MODEL_DIR / 'scaler_v15.joblib')
joblib.dump(feature_cols, MODEL_DIR / 'feature_cols_v15.joblib')

# Save config
config = {
    'timeframes': TIMEFRAMES,
    'tp_pips': TP_PIPS,
    'sl_pips': SL_PIPS,
    'forward_bars': FORWARD_BARS
}
joblib.dump(config, MODEL_DIR / 'config_v15.joblib')

print(f"Models saved to {MODEL_DIR}")